### DeepDream
* 反向运行一个卷及神经网络：对卷及神经网络输入做梯度上升，以便将卷及神经网络靠顶部的某一层的某个过滤器激活最大化

In [2]:
from keras.applications import inception_v3
from keras import backend as K
# from keras.models import load_model

# model = load_model('inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5')

# 禁用所有预训练有关的操作
K.set_learning_phase(0)

# 构建不包括全连接层的InceptionV3网络，使用与训练的Inception网络权重加载模型
model = inception_v3.InceptionV3(weights='imagenet', include_top=False)

Using TensorFlow backend.


In [3]:
model.summary()

Model: "inception_v3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, None, None, 3 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, None, None, 3 96          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, None, None, 3 0           batch_normalization_1[0][0]      
_______________________________________________________________________________________

### 设置DeepDream配置
* 这一步是计算损失，在梯度上升过程中需要最大化的量，这里我们将多个层的所有过滤器的激活同时最大化，选择不同的层可视化结果会很不一样

In [4]:
# 下列字典将层的名称映射为一个系数，这个系数定量表示该层激活对你要最大化的损失的贡献大小。
layer_contributions = {
    'mixed2' : 0.2,
    'mixed3' : 3.,
    'mixed4' : 2.,
    'mixed5' : 1.5,
}

### 定义最大化损失

In [10]:
# 创建一个字典，将层的名称映射为层的实例
layer_dict = dict([(layer.name, layer) for layer in model.layers])

# 在定义损失时将层的贡献添加到这个标量变量中
loss = K.variable(0.)
for layer_name in layer_contributions:
    coeff = layer_contributions[layer_name]
    # 获取层的输出
    activation = layer_dict[layer_name].output
    
    scaling = K.prod(K.cast(K.shape(activation), 'float32'))
    # 将该层特征的L2范数添加到loss，为了避免边界伪影，损失仅包含非边界的像素
    loss += coeff * K.sum(K.square(activation[:, 2: -2, 2: -2, : ])) / scaling
    

RuntimeError: Variable += value not supported. Use variable.assign_add(value) to modify the variable value and variable = variable + value to get a new Tensor object.

### 梯度上升过程

In [9]:
# 这个张量用于保存生成的图像，即梦境图像
dream = model.input

# 计算损失相对于梦境图像的梯度
grads = K.gradients(loss, dream)[0]

# 将梯度标准化
grads /= K.maximum(K.mean(K.abs(grads)), 1e-7)

# 给定一张输出图像，设置一个keras函数来获取损失值和梯度值
outputs = [loss, grads]
fetch_loss_and_grads = K.function([dream], outputs)
    
def eval_loss_and_grads(x):
    outs = fetch_loss_and_grads([x])
    loss_value = outs[0]
    grad_values = outs[1]
    return loss_value, grad_values

def gradient_ascent(x, iterstions, step, max_loss=None):
    for i in range(iterations):
        loss_value, grad_values = eval_loss_and_grads(x)
        if max_loss is not None and loss_value > max_loss:
            break
        print('....Loss value at', ':', loss_value)
        x += step * grad_values
    return x

ValueError: Attempt to convert a value (None) with an unsupported type (<class 'NoneType'>) to a Tensor.

### 在多个连续尺度上运行梯度上升

In [ ]:
import numpy as np

# 改变一下超参数可以得到新的效果
# 梯度上升的步长
step = 0.01   
# 运行梯度上升的尺度个数
num_octave = 3
# 两个尺度之间的大小比例
octave_scale = 1.4
# 在每个尺度上运行梯度上升的步数
iterations = 20

#如果增大损失到10，要中断梯度上升的过程，以避免丑陋的伪影
mox_loss = 10.

# 读取图像
base_image_path = '/home/wukong/data/test1.jpg'


# 将基础图像加载成一个Numpy数组
def preprocess_image(image_path):
    img = image.load_img(image_path)
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = inception_v3.preprocess_input(img)
    return img

img = preprocess_image(base_image_path)

original_shape = img.shape[1:3]
# 准备一个由形状元组成的列表，它定义了运行梯度上升的不同尺度
successive_shapes = [original_shape]
for i in range(1, num_octave):
    shape = tuple([int(dim / (octave_scale ** i))
        for dim in original_shape])
    successive_shape.append(shape)
    
    # 将形状列表反转，变为升序
    successive_shapes = successive_shapes[::-1]
    
    # 将图像Numpy数组的大小缩放到最小尺寸
    original_img = np.copy(img)
    shrunk_original_img = resize_img(img, successive_shapes[0])
    
    for shape in successive_shapes:
        print(;Processing image shape), shape
        # 将梦境图像放大
        img = resize_img(img, shape)
        # 运行梯度上升改变梦境图像
        img = gradient_ascent(img, iterations=iterations, step=step, max_loss=max_loss)
        
        # 将原始图像的较小版本放大，他会变得像素化
        upscaled_shrunk_original_img = resize_img(shrunk_original_img, shape)
        # 在这个尺寸上计算呢原始图像高质量版本
        same_size_original = resize_img(original_img, shape)
        # 二者的差别就是放大过程中丢失的细节
        lost_detail = same_size_original - upscaled_shrunk_original_img
        
        # 将丢失的细节重新注入到梦境图像中
        img += lost_detail
        shrunk_original_img = resize_img(original_img, shape)
        save_img(img, fname='dream_at_scale_' + str(shape) + '.png')
        
    save_img(img, fname='final_fream.png')

### 辅助函数

In [ ]:
import scipy
from keras.preprocessing import image

def resize_img(img, size):
    img = np.copy(img)
    factors = (1,
               float(size[0] ) / img.shape[1],
               float(size[1])  / img.shape[2],
               1)
    return scipy.ndimage.zoom(img, factors, order=1)
def save_img(img, fname):
    pil_img = deprocess_image(np.copy(img))
    scipy.misc.imsave(fname, pil_img)

def deprocess_image(x):
    if K.image_data_format() == 'channels_first':
        x = x.reshape((3, x.shape[2], x.shape[3]))
        x = x.transpose((1, 2, 0))
    else:
        # 对模型所做的预处理进行反向操作
        x = x.reshape((x.shape[1], x.shape[2], 3))
    x /= 2.
    x += 0.5
    x *= 255.
    x = np.clip(x, 0, 255).astype('uint8')
    return x